# Covid 19 Chest X-Ray Analysis with GradCam

In this Notebook I use a pretrained `densenet121` for analysing Covid-19 Xrays and Visualise them using GradCam algorithm.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from skimage import io, transform
from torch.utils.data import Dataset
import torch
import os
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from PIL import Image,ImageOps
import torchvision
from torch import optim,nn
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tqdm
import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from util_grad import *

## Read  and  Process  Data

In [ ]:
## Define transfoms
standard_normalization = transforms.Normalize((0.5), (0.5))
data_transforms = transforms.Compose([transforms.RandomResizedCrop(256),
                                     transforms.ToTensor(),
                                     standard_normalization])


In [ ]:
# Read Data
dict_ = {0:"Covid",1:"Lung_Opacity",2:"Normal",3:"Viral Pheumonia"}

df = pd.read_csv("../input/processed-csv/data.csv")
df = pd.concat([df,pd.get_dummies(df.tag)], axis=1)


In [ ]:
#make train, test and val set

train, test = train_test_split(df,test_size=.2)
train_, val = train_test_split(train,test_size=.2)

In [ ]:
#reseting index
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
train_ = train_.reset_index(drop=True)
val = val.reset_index(drop=True)

In [ ]:
#make dataloader
root_dir = "../input/covid19-radiography-database/COVID-19_Radiography_Dataset"

trainset = Xray_dataset(train,root_dir,transform = data_transforms)
train_loader = DataLoader(trainset, batch_size=32, shuffle=True)

testset = Xray_dataset(test,root_dir,transform = data_transforms)
test_loader = DataLoader(testset, batch_size=32, shuffle=True)

trainset_ = Xray_dataset(train_,root_dir,transform = data_transforms)
train_loader_ = DataLoader(trainset_, batch_size=32, shuffle=True)

valset = Xray_dataset(val,root_dir,transform = data_transforms)
val_loader = DataLoader(valset, batch_size=32, shuffle=True)

use_cuda = torch.cuda.is_available()
num_classes = 4

loaders = {"train": train_loader_,
           "test": test_loader,
           "val": val_loader}

## Download and Prepare Neural Network

In [ ]:
model = torchvision.models.densenet121(pretrained = True, progress = True) 
model.classifier = nn.Linear(model.classifier.in_features, num_classes)
for param in model.features.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True
if use_cuda:
    model = model.cuda()

In [ ]:
## Define Loss and optim
optimizer = optim.SGD(model.classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

## Train Model

In [ ]:
## Train
train_model(20, loaders, model, optimizer, criterion, use_cuda, 'model_transfer.pt');

## Load the Saved Weights

In [ ]:
import torch

In [ ]:
model = torch.load("./model_transfer.pt")

## Test Model

In [ ]:
test_model(loaders,model,criterion,use_cuda)
conf_met(model,test_loader)

## Draw GradCam Output

Grad-CAM: Gradient-weighted Class Activation Mapping

In [ ]:
gradcam(model,testset[3][0].unsqueeze(0).cuda())